In [1]:
!nvidia-smi

Sun Jul 16 07:52:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:4F:00.0 Off |                    0 |
| 30%   27C    P8    22W / 300W |      3MiB / 45631MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:52:00.0 Off |                    0 |
| 30%   

### Preprocessing Rule
1. extract talk id and (content + file id) separately
2.  Check the length of both
    - If same length:
        - Concatenate both
    - If different length:
        - Delete duplicates in content df or add corresponding talkid refer to other languages.  
        - When same length, concatenate both. 
3. Sort the concatenated df by talkid
4. Check if the parallelism work in two languages 
5. Delete row where the other language file doesn't have talk id

In [17]:
from datasets import load_dataset
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [196]:
languages = ["ja", "en", "de", "ko", "zh", "zh-cn", "zh-tw", "ar", "fr"]

def create_talkid_df(languages):
    talkid_dfs = {}
    print ("The number of talk ids in each languages: ")
    for lang in languages:
        with open(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_{lang}.xml', 'r', encoding="utf-8") as f: # , 
            data = f.read()
            bs_data = BeautifulSoup(data, "xml")
            talkid = bs_data.find_all("talkid")
            print (f"{lang}:", len(talkid))
            talkid_df = pd.DataFrame({"talk_id":talkid})
            talkid_df["talk_id"]=talkid_df["talk_id"].map(lambda x:str(x))
            talkid_df["talk_id"]=talkid_df["talk_id"].map(lambda x:x[8:-9]) # Extract only the ids from data
            talkid_dfs[f"{lang}"] = talkid_df
    return talkid_dfs
        
talkid_dfs = create_talkid_df(languages)

The number of talk ids in each languages: 
ja: 2489
en: 2683
de: 2161
ko: 2565
zh: 163
zh-cn: 2429
zh-tw: 2578
ar: 2545
fr: 2597


In [320]:
talkid_df_ja = talkid_dfs["ja"]
talkid_df_ja
talkid_df_de = talkid_dfs["de"]
talkid_df_de
talkid_df_en = talkid_dfs["en"]
talkid_df_en
talkid_df_ko = talkid_dfs["ko"]
talkid_df_ko
talkid_df_zh_cn = talkid_dfs["zh-cn"]
talkid_df_zh_cn
talkid_df_ar = talkid_dfs["ar"]
talkid_df_ar
talkid_df_fr = talkid_dfs["fr"]

In [362]:
with open(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_de.xml', 'r', encoding="utf-8") as f: # , 
            data = f.read()
            bs_data = BeautifulSoup(data, "xml")
for e in bs_data.find_all('head'):
    if e.find('talkid') == None:
        #if len(e.find{'content'}) > 1: 
        print(e)

<head>
<url>http://www.ted.com/talks/roger_antonsen_math_is_the_hidden_secret_to_understanding_the_world?language=de</url>
<encoding>UTF-8</encoding>
<content-type>text/html; charset=utf-8</content-type>
<description>TED Talk Subtitles and Transcript: Entschlüsseln Sie die Geheimnisse und das Innenleben der Welt mithilfe einer der fantasievollsten Kunstformen: Mathematik. Roger Antonsen erläutert, wie ein leichter Perspektivenwechsel Muster, Zahlen und Formeln aufdecken und als Zugang zu Empathie und Verständnis dienen kann.</description>
<title>Roger Antonsen: Mathematik: das verborgene Geheimnis zum Verständnis der Welt</title>
<transcription>
<seekvideo id="1">Hallo. Ich möchte über Verständnis, die Eigenart und das Wesen von Verständnis sprechen, denn wir alle bemühen uns, zu verstehen. Wir wollen Dinge verstehen. Ich behaupte, dass Verständnis mit der Fähigkeit zusammenhängt, die Perspektive zu wechseln. Wenn man das nicht hat, kann man nicht verstehen. Das behaupte ich jedenfalls

In [68]:
def create_content_df(languages):
    content_dfs = {}
    print ("The number of content data in each languages: ")
    for lang in languages:
        content_df = pd.read_xml(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_{lang}.xml', parser='lxml')
        content_df = content_df.drop(["head", "id"], axis = 1)
        print (f"{lang}: ", len(content_df))
        content_dfs[f"{lang}"] = content_df
    return content_dfs
    
content_dfs = create_content_df(languages)

The number of content data in each languages: 
ja:  2490
en:  2683
de:  2163
ko:  2565
zh:  163
zh-cn:  2429
zh-tw:  2578
ar:  2545
fr:  2598


In [198]:
content_df_ja = content_dfs["ja"]
content_df_ja 

,content
0,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...
2485,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [322]:
content_df_ja = content_dfs["ja"]
content_df_ja
content_df_de = content_dfs["de"]
content_df_de
content_df_en = content_dfs["en"]
content_df_en
content_df_ko = content_dfs["ko"]
content_df_ko
content_df_zh_cn = content_dfs["zh-cn"]
content_df_zh_cn
content_df_ar = content_dfs["ar"]
content_df_ar
content_df_fr = content_dfs["fr"]

In [340]:
def concatenate_df(df1, df2, df1_col, df2_col):
    
    if df1.isnull().any()[df1_col]:
        print ("NAN for df1")
        df1[df1_col] = df1[df1_col].replace({np.NaN: "no contents"})
        print (df1.isnull().any(), "for df1")
        print (df1)
        
    if df2.isnull().any()[df2_col]:
        print ("NAN for df2")
        df2[df2_col] = df2[df2_col].replace({np.NaN: "no contents"})
        
        
    if len(df1) == len(df2): 
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())

        
    elif len(df1) > len(df2):
        #print ("(df1.duplicated() == False)", df1[df1[df1_col].duplicated() == False], "df1[df1_col]=='no contents'", df1[df1[df1_col]=="no contents"])
        df1 = df1[(df1[df1_col].duplicated() == False) | (df1[df1_col]=="no contents")]
        print ("df2for NAN", df2.isnull().any()[df2_col])
        print ("df1for NAN", df1.isnull().any()[df1_col])
        print (df1, df2)
        print ("df1.duplicated", df1[df1["content"].duplicated()])
        df1.reset_index(drop=True, inplace = True)
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())
    
    """
    elif len(df1) < len(df2):
        df2 = df2[(df2[df2_col].duplicated() == False) & (df2[df2_col].isnull()==False)] # Remove the duplicated content 
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())
    """
    
    return df1, df2, concat_df



In [327]:
_, _, concat_df_ja = concatenate_df(content_df_ja, talkid_df_ja, "content", "talk_id")
concat_df_ja

df2for NAN False
df1for NAN False
                                                content
0     人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1     胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2     皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3     （ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4     文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...                                                 ...
2485  (拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486  おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487  どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488  (♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...
2489  ご来場の皆さん 本日はありがとうございます 皆さんなら 長期的視野に立った都市開発が 人類を...

[2489 rows x 1 columns]      talk_id
0      12908
1      13062
2      11327
3      12459
4      12346
...      ...
2484      96
2485      66
2486       1
2487       7
2488      53

[2489 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...,12908
1,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...,13062
2,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...,11327
3,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...,12459
4,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...,12346
...,...,...
2484,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...,96
2485,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...,66
2486,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...,1
2487,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...,7


In [328]:
_, _, concat_df_ko = concatenate_df(content_df_ko, talkid_df_ko, "content", "talk_id")
concat_df_ko

content    False
talk_id    False
dtype: bool


,content,talk_id
0,"""들어보렴. 들리니?"" 제 할머니가 물으셨죠. ""들어보렴. 딱정벌레가 뭐라고 하는지...",13591
1,마름병이 밤나무를 멸종시킬까요? 농부들은 대부분 아니라고 생각합니다. 마름병은 뿌리...,13062
2,"최근 우리는 사이버 공격이 비즈니스 세계에 미치는 영향을 봤습니다. JP 모건, 야...",12501
3,(음악)\n(노래) 오오 오오\n그녀는 부자가 되고 싶어요\n그녀는 유명해지고 싶죠...,12803
4,"2014년 6월 13일 평소와 같은 금요일 라이베리아의 수도, 몬로비아의 구원 병원...",12302
...,...,...
2560,(박수) 감사합니다. 저는 지금 기쁘기도 하면서 새로운 도전을 하고있는 기분이라고 ...,96
2561,안녕하세요. 정말 좋지 않습니까? 강연들이 너무 좋아 정신을 못 차리겠네요. 사실 ...,66
2562,"감사합니다, 크리스. 이곳에 두 번이나 설 기회를 얻게된 것을 큰 영광으로 생각합니...",1
2563,"♫안녕 자동응답기, 나의 오랜 친구여 (웃음) 기술 지원이 필요해서 또 전화했어. ...",7


In [341]:
_, _, concat_df_de = concatenate_df(content_df_de, talkid_df_de, "content", "talk_id")
concat_df_de # After 924, which is index 1475, Lassen Sie~ doesnt have talkid

df2for NAN False
df1for NAN False
                                                content
0     Ich wuchs auf als weißer Mann, säkular, in ein...
1     Treppen sind wahrscheinlich eine der emotional...
2     Es gibt keine schlechen Knöpfe, bloß schlechte...
3     Ich bin Wissenschaftlerin, und ein großer Star...
4     Wissen Sie, auf wen ich neidisch bin? Auf Leut...
...                                                 ...
2158  (Applaus) Vielen Dank. Ich muss Ihnen sagen, i...
2159  Guten Morgen. Wie geht es Ihnen?\n(Lachen)\nEs...
2160  Vielen Dank, Chris. Es ist mir wirklich eine E...
2161  ♪ Hallo Sprachbox, alter Freund ♪ (Lachen) ♪ I...
2162  Wenn Sie heute hier sind, und ich freue mich, ...

[2162 rows x 1 columns]      talk_id
0      12303
1      12352
2      12351
3       9930
4      10550
...      ...
2156      96
2157      66
2158       1
2159       7
2160      53

[2161 rows x 1 columns]
df1.duplicated           content
826   no contents
1055  no contents
content    False
ta

,content,talk_id
0,"Ich wuchs auf als weißer Mann, säkular, in ein...",12303
1,Treppen sind wahrscheinlich eine der emotional...,12352
2,"Es gibt keine schlechen Knöpfe, bloß schlechte...",12351
3,"Ich bin Wissenschaftlerin, und ein großer Star...",9930
4,"Wissen Sie, auf wen ich neidisch bin? Auf Leut...",10550
...,...,...
2157,"(Applaus) Vielen Dank. Ich muss Ihnen sagen, i...",66
2158,Guten Morgen. Wie geht es Ihnen?\n(Lachen)\nEs...,1
2159,"Vielen Dank, Chris. Es ist mir wirklich eine E...",7
2160,"♪ Hallo Sprachbox, alter Freund ♪ (Lachen) ♪ I...",53


In [330]:
_, _, concat_df_fr = concatenate_df(content_df_fr, talkid_df_fr, "content", "talk_id")
concat_df_fr

df2for NAN False
df1for NAN False
                                                content
0     Je suis un immigrant venu de l'Ouganda vivant ...
1     [Provocation de Danny Hillis :]\n[Il est grand...
2     Mon voyage pour m'éloigner de l'extrémisme vio...
3     « Le mildiou tuera-t-il le châtaignier ? » Les...
4     Il est 4 heures du matin. Je me réveille dans ...
...                                                 ...
2593  (Applaudissements) Merci. Je dois vous dire qu...
2594  Bonjour. Comment allez-vous? C'était bien, n'e...
2595  Merci beaucoup, Chris. C'est vraiment un honne...
2596  Bonjour répondeur, mon vieil ami. (Rires) J'ap...
2597  Si vous êtes là aujourd'hui — et j'en suis trè...

[2598 rows x 1 columns]      talk_id
0      14392
1      12799
2       8298
3      13062
4       9982
...      ...
2592      96
2593      66
2594       1
2595       7
2596      53

[2597 rows x 1 columns]
content    False
talk_id     True
dtype: bool


,content,talk_id
0,Je suis un immigrant venu de l'Ouganda vivant ...,14392
1,[Provocation de Danny Hillis :]\n[Il est grand...,12799
2,Mon voyage pour m'éloigner de l'extrémisme vio...,8298
3,« Le mildiou tuera-t-il le châtaignier ? » Les...,13062
4,Il est 4 heures du matin. Je me réveille dans ...,9982
...,...,...
2593,(Applaudissements) Merci. Je dois vous dire qu...,66
2594,"Bonjour. Comment allez-vous? C'était bien, n'e...",1
2595,"Merci beaucoup, Chris. C'est vraiment un honne...",7
2596,"Bonjour répondeur, mon vieil ami. (Rires) J'ap...",53


In [331]:
_, _, concat_df_ar = concatenate_df(content_df_ar, talkid_df_ar, "content", "talk_id")
concat_df_ar

content    False
talk_id    False
dtype: bool


,content,talk_id
0,هل حصل وذكرتَ دراسةً ما كمرجع لكلامك وأنت في ا...,12908
1,يقول الناس أنني شخصاً لطيفاً... إلى الحد حيثما...,12354
2,أنحدر من عائلة مكونة من خمسة إخوة، جميعهم علما...,3618
3,"أتذكر أنني فكرت -لنفسي- ، ""هذا سيغير كل شيء عن...",12353
4,أعتقد أن الدرج قد يكون واحدًا من العناصر الماد...,12352
...,...,...
2540,(تصفيق) شكراً. علي أن أخبركم بأنني أشعر بالتحد...,96
2541,صباح الخير. كيف حالكم؟ كان يوما عظيما، أليس كذ...,66
2542,شكرا جزيلا كريس. انه فعلا شرف عظيم لي ان أصعد ...,1
2543,أهلا بك صديقي العزيز، البريد الصوتي♪ (ضحك) لقد...,7


In [332]:
_, _, concat_df_zh_ch = concatenate_df(content_df_zh_cn, talkid_df_zh_cn, "content", "talk_id")
concat_df_zh_ch

NAN for df1
content    False
dtype: bool for df1
                                                content
0     我自认为既是一位艺术家， 又是一位设计师。 我在一个研究 人工智能的实验室工作。 我们正在尝...
1     我是一个编剧兼导演 写一些社会变革的故事， 因为我相信故事能够 打动并驱动我们。 故事使我们...
2     我记得曾经这样想过， “这将会颠覆我们交流的方式。”\n【小发明】 【大点子】\n【超链接:...
3     我认为台阶可能是 建筑家需要去解决的在情感上 最具可塑性的物理元素之一。\n[小事物，大想法...
4     没有不好的扣子，只有不好的人。 那听起来真么样？可以吗？\n“小物件” “大想法”\n“Is...
...                                                 ...
2424  （掌声） 谢谢。 我必须告诉你们我同时感到挑战与兴奋。 让我感到兴奋的是，我有一个付出回馈的...
2425  早上好，还好吗? 很好吧，对不对? 我已经飘飘然了! 我要飘走了。(笑声) 这次会议有三个主...
2426  非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。 这个会议真是...
2427  语音信箱 我的老朋友♪ (笑声) 我又打了技术支持电话♪ 我没听老板的警告 在周一早上打的♪...
2428  假若你今天有幸来此—我对你的到来亦是甚喜－－ 你们一定听说过可持续发展可以拯救我们的生活 然...

[2429 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,我自认为既是一位艺术家， 又是一位设计师。 我在一个研究 人工智能的实验室工作。 我们正在尝...,12498
1,我是一个编剧兼导演 写一些社会变革的故事， 因为我相信故事能够 打动并驱动我们。 故事使我们...,11997
2,我记得曾经这样想过， “这将会颠覆我们交流的方式。”\n【小发明】 【大点子】\n【超链接:...,12353
3,我认为台阶可能是 建筑家需要去解决的在情感上 最具可塑性的物理元素之一。\n[小事物，大想法...,12352
4,没有不好的扣子，只有不好的人。 那听起来真么样？可以吗？\n“小物件” “大想法”\n“Is...,12351
...,...,...
2424,（掌声） 谢谢。 我必须告诉你们我同时感到挑战与兴奋。 让我感到兴奋的是，我有一个付出回馈的...,96
2425,早上好，还好吗? 很好吧，对不对? 我已经飘飘然了! 我要飘走了。(笑声) 这次会议有三个主...,66
2426,非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。 这个会议真是...,1
2427,语音信箱 我的老朋友♪ (笑声) 我又打了技术支持电话♪ 我没听老板的警告 在周一早上打的♪...,7


In [333]:
_, _, concat_df_en = concatenate_df(content_df_en, talkid_df_en, "content", "talk_id")
concat_df_en

NAN for df1
content    False
dtype: bool for df1
                                                content
0     I will never forget the first time I visited a...
1     Don't you love a good nap?\n(Laughter)\nJust s...
2     So, I'll be speaking to you using language ......
3     All right, let's get ready for the worst TED T...
4     [Ojibwe: Hello. My English name is Tara; my Na...
...                                                 ...
2678  Thank you. I have to tell you I'm both challen...
2679  Good morning. How are you?\n(Laughter)\nIt's b...
2680  Thank you so much, Chris. And it's truly a gre...
2681  (Music: "The Sound of Silence," Simon & Garfun...
2682  If you're here today — and I'm very happy that...

[2683 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,I will never forget the first time I visited a...,14471
1,Don't you love a good nap?\n(Laughter)\nJust s...,13022
2,"So, I'll be speaking to you using language ......",12727
3,"All right, let's get ready for the worst TED T...",13778
4,[Ojibwe: Hello. My English name is Tara; my Na...,12800
...,...,...
2678,Thank you. I have to tell you I'm both challen...,96
2679,Good morning. How are you?\n(Laughter)\nIt's b...,66
2680,"Thank you so much, Chris. And it's truly a gre...",1
2681,"(Music: ""The Sound of Silence,"" Simon & Garfun...",7


# Pandas Dataframe

In [7]:
from datasets import load_dataset
import pandas as pd
from bs4 import BeautifulSoup

In [162]:
df_ja = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_ja.xml', parser='lxml')

In [163]:
df_ja

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [59]:
df_ja["content"].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2485    False
2486    False
2487    False
2488    False
2489    False
Name: content, Length: 2490, dtype: bool

In [101]:
df_ja[df_ja["content"].duplicated()]


,id,head,content
699,701,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...
1494,1497,NaN,None
1532,1536,NaN,None
1594,1598,NaN,None
1623,1627,NaN,None
1686,1690,NaN,None
1731,1735,NaN,None
1767,1771,NaN,None
2169,2177,NaN,None
2350,2359,NaN,None


In [171]:
df_ja

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [180]:
#df_ja[df_ja["content"].duplicated() == False]

df_removed = df_ja[(df_ja["content"].duplicated() == False) | (df_ja["content"].isnull()==True)]
df_removed
#df_ja[(df_ja["content"] != np.NaN)]
#df_ja[df_ja["content"].duplicated()]
#df_ja[df_ja["content"].isnull()]
#df_ja = df_ja.drop(df_ja[df_ja[df1_col].duplicated()!="no contents"], axis=0)
#df_ja[df_ja["content"].isnull()==False]

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [29]:
df_ja[698:700]
        

,id,head,content
698,700,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...
699,701,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...


File id 700 does not have talk id, so delete file 700 and all the empty content

In [15]:
df_ja[902:905]

,id,head,content
902,904,NaN,今日 皆さんにお話したいのは 私たちのあり方を決める パワフルで 基礎的なもの― 「声」につ...
903,905,NaN,None
904,906,NaN,まず 皆さんがスーパーにいると 想像して下さい 食料品を買うと 選択肢が与えられます プラス...


In [16]:
df_ja = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_ja.xml', parser='lxml')
df_ja= pd.concat([talk_id_ja, df_ja], axis=1)
#df_ja["talk_id"][:-1] = df_ja["talk_id"][:-1].map(lambda x:int(x))
df_ja['talk_id'].isnull()
df_ja

,talk_id,id,head,content
0,12908,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,13062,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,11327,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,12459,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,12346,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...,...
2485,66,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,1,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,7,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,53,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [17]:
df_en = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_en.xml', parser='lxml')
df_en= pd.concat([talk_id_en, df_en], axis=1)
#df_en["talk_id"]= df_en["talk_id"].map(lambda x:int(x))
df_en[df_en["content"].isnull()]
df_en[df_en["id"].isnull()]

,talk_id,id,head,content


In [25]:
df_en["content"][902:905]

902    I came here to show you the Fotokite. It's a t...
903    Let me tell you a story about a little girl na...
904    George and Charlotte Blonsky, who were a marri...
Name: content, dtype: object

In [92]:
df_ja["content"][2]

'皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成果が際立っていればいるほど 私たちは その人を天才と呼んだり もしかすると他の惑星から来た 異星人に違いないなどと言い 自分とは まったく違う存在だと 思いがちです でも そうなのでしょうか？\n１つの例から始めましょう ニュートンのリンゴの話は 知っていますよね？ 本当なのでしょうか？ 多分違うでしょう それでも 「リンゴ」が何もなかったとは 考えにくいです つまり 何か足掛かりや 特定の状況があって 万有引力の着想が不可能ではない 状態にあったということです そして間違いなく ニュートンにとっては 不可能ではありませんでした 可能だったのです それも 何かの理由で ある時点において それはリンゴのように 容易に取れるものだったのです それこそがリンゴなのです\nアインシュタインはどうでしょう？ 相対性理論は アイデアの歴史における もうひとつの大きな飛躍であり 彼以外には 着想できない ものだったのでしょうか？ あるいはむしろ それは手の届くところにある 可能なものだったのか— もちろんアインシュタインにとっては ということですが— それは彼独自の科学の道における 小さなステップだったのか？ もちろん私たちには そのような道は見出せないでしょうが かと言って そんな道が 存在しないとは言い切れません\nこれは とても 示唆には富んでいますが まだ確かとは言い難いでしょう 素晴らしいアイデアがどこから来るのか あるいは 一般に 新しいものが いかにもたらされるのか 知りたいと思うのであれば 物理学者として 科学者として 私が学んだことは 解の半分は 適切な問いを 立てることにあるということですが 私たちは今や 適切な問いを 見付け 取り組むための 素晴らしい概念的な枠組みを 手にしつつあると思います そこで既知の領域の境界まで 皆さんをお連れして— まあ 私が知る 限りの境界ですが 既知の事柄が 強力で素晴らしい出発点となり 新規性や イノベーション 創造性といった言葉の 深い意味の把握に繋がることを 示しましょう\n「新しいもの」について議論しますが もちろん その背後の 科学にも触れます 新しいものが私たちの生活に 取り込まれる方法は色々あり それは ごく個人的なもの

In [66]:
pd

NameError: name 'df_ja' is not defined

In [74]:
for doc_id, doc in enumerate (data["content"][0:100]):
    doc_input = doc.split("\n")
    
    print (doc_input[0:5])
    

<class 'str'>
['人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？', '（笑）', 'この間 友人とコーヒーを飲みながら こんな話をしました 「最近の研究を “読んだ”のだけど コーヒーは女性が鬱になるリスクを 下げてくれるんだってね」 でも本当は ツイートを読んだだけです', '（笑）', 'そのツイート曰く']
<class 'str'>
['胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り出し続ける 別の寄生者が現れ 胴枯病を絶えさせるまで [ロバート・フロスト 1936年]', '20世紀初めに 40億本もあった アメリカ東海岸の栗の木が 菌類感染のため 壊滅しました 経済的に重要な作物も含めた 植物にとって 菌類は 最も破壊的な 病原体です 真菌感染症による 農作物の損失は 世界で年に 数十億ドル規模にもなるなんて 想像できますか？ カロリーにして ５億人を食べさせるに足る量です これは大きな影響を もたらします 発展途上国における飢饉 農家や流通業者の 収入の大幅な減少 消費者にとっての 価格の高騰 菌類が作る毒素 マイコトキシンにさらされるリスク', '私たちが直面している問題は そういう恐ろしい病気を防ぎ 処置するための 現在の方法は 自然の抵抗力を利用する 遺伝的防除にせよ 輪作や 種子処理 その他のものにせよ 限定的 短期的だ ということです 絶えず繰り返さなければ なりません もっと効率的な方法を 速やかに開発する必要があり そのためにも 新しい防かび法が 狙いを付けられる生物学的メカニズムを 見極める必要があるのです', '菌類の特徴の１つは 移動はできず 伸びて成長するだけで 菌糸体と呼ばれる高度な網状の構造を 形成するということです 1884年に 植物病理学の父である アントン・ド・バリーは 菌類は 宿主が出す シグナルによって 導かれるのではと 考えました 宿主というのは 菌類が住み着く植物のことです つまりこのシグナルが 灯台の役割を果たし 菌類は 植物を見付け そちらへと成長し 到達し 侵入して支配してしまう ということです そのようなシグナルを 発見することは 菌類と植物の間のやり取りを 阻害する方法を開発するための鍵になると バリ

Question:
- Can we sort xml file by talk_id? or can we extract talk_id info in pandas too?  